# Example notebook for loading TOFWERK HD5 dataset

Load the FIB image and peak data

In [ ]:
import twtof
filename = "./data/20230722_magnetotactic_HO027_1_Au_pos_spot2_run1_30kV_50pA.h5"
fib, mass, peak = twtof.imread(filename)


Interactive display of the image and the associated spectrum

In [ ]:
%matplotlib ipympl
from ipywidgets import interact
import matplotlib.pyplot as plt
fig, ax = plt.subplots(1,3,figsize=(10,5))
axfib = ax[0].imshow(fib[0].squeeze())
crosshair = ax[0].plot(0,0,'w+',ms=50)
axpeak = ax[1].plot(peak[0,0,0].ravel())
sline = ax[1].plot((0,0),(0,2),'r-')
axmap = ax[2].imshow(peak[0,:,:,0].squeeze())
ax[1].set(xlabel='mass', ylabel='count')
ax[1].set_box_aspect(1)
ax[0].set_axis_off()
ax[2].set_axis_off()

@interact(
        x = (0,fib.shape[2]-1, 1), 
        y = (0,fib.shape[1]-1, 1), 
        z = (0, fib.shape[0]-1, 1), 
        c = (0,peak.shape[3]-1, 1),
        cmap = ['viridis','jet','gray'])
def display_twotof(x,y,z,c,cmap):
    axfib.set(data = fib[z], cmap=cmap)
    crosshair[0].set(data=([x],[y]))
    ax[0].set_title(f'FIB Image\n({x},{y},{z})')
    d = int(fib.shape[1]/peak.shape[1])    
    axpeak[0].set(data = (mass, peak[z,y//d,x//d].ravel()))
    sline[0].set(data=((c,c),(0,2)))
    ax[1].set_title(f'mass {c}')
    axmap.set(data = peak[z,:,:,c].squeeze(), cmap=cmap)
    ax[2].set_title('Peak data')

Load the peak table

In [ ]:
with twtof.TofH5Reader(filename) as f:
    peak_df = f.load_peak_table()
peak_df

Load all the data in a dictionnary

In [ ]:
data = twtof.as_dict(filename)
data['Acquisition log']

# Export FIB image and peak to TIF

In [ ]:
import twtof
import tifffile
import numpy as np
filename = "./data/20230722_magnetotactic_HO027_1_Au_pos_spot2_run1_30kV_50pA.h5"
fib, mass, peak = twtof.imread(filename)
tifffile.imwrite('mass.tif', np.moveaxis(peak,[0,3,1,2],[0,1,2,3]), imagej=True, metadata={'axes': 'ZCYX'})
tifffile.imwrite('fib.tif', fib.astype(np.uint16), imagej=True, metadata={'axes': 'ZYX'})

# Average spectra in ROI

In [ ]:
lbl = multi_class_segmenter.mask[::8,::8]
plt.plot(peak[0,lbl==1,:].mean(axis=0))

In [ ]:
from mpl_interactions import image_segmenter
import ipywidgets as widgets

fig, ax = plt.subplots(1,2)
    
N_classes = 3

slice_selector = widgets.IntSlider(value=0, min=0, max=fib.shape[0]-1, description="Slice")

class_selector = widgets.Dropdown(options=list(range(1, N_classes + 1)), description="ROI")


def update(change):
    multi_class_segmenter.current_class = class_selector.value
    multi_class_segmenter.erasing = erasing_button.value    
    multi_class_segmenter.displayed.set_data(fib[slice_selector.value])       
    
    d = fib.shape[1] // peak.shape[1]        
    lbl = multi_class_segmenter.mask[::d,::d].astype(int)
    for c in np.unique(lbl)[1:]:            
        f = peak[slice_selector.value,lbl==c,:].mean(axis=0)
        axpeak[c-1][0].set_data(mass, 0.01+f)           
    refresh_btn.value = False
    print('x')


erasing_button = widgets.Checkbox(value=False, description="Erasing")
refresh_btn = widgets.Button(value=False, description="Refresh")
erasing_button.observe(update, names="value")
class_selector.observe(update, names="value")
slice_selector.observe(update, names="value")
refresh_btn.on_click(update)

axpeak = [ax[1].semilogy([0,300],[0.01,0.01]) for k in range(N_classes)]
ax[1].set_box_aspect(1)
ax[1].set(xlim=(0, 300), ylim=(0.01, 2))
multi_class_segmenter = image_segmenter(fib[0], nclasses=3, mask_alpha=0.76, ax=ax[0])
multi_class_segmenter.ax
display(widgets.HBox([slice_selector, erasing_button, class_selector,refresh_btn]))


In [ ]:
d = fib.shape[1] // peak.shape[1]        
lbl = multi_class_segmenter.mask[::d,::d].astype(int)
for c in np.unique(lbl)[1:]:            
    f = peak[slice_selector.value,lbl==c,:].mean(axis=0)
    axpeak[c-1][0].set_data(mass, np.log(0.1+f))             

In [ ]:
len(axpeak)